[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MouseLand/rastermap/blob/main/notebooks/rastermap_singleneurons.ipynb)

# Rastermap sorting of 137 neurons

We will use a processed version of the data from [Grosmark & Buzsaki, 2016](https://crcns.org/data-sets/hc/hc-11/about-hc-11). 137 neurons from rat hippocampal CA1 were recorded over several hours using eight bilateral silicon-probes. We selected the 33 minute period from the recording in which the rat traverses a linear track of length 1.6 meters. We binned the neural activity into time bins of length 200 ms. 

First we will install the required packages, if not already installed. If on google colab, it will require you to click the "RESTART RUNTIME" button because we are updating numpy.

In [ ]:
!pip install numpy>=1.24 # (required for google colab)
!pip install rastermap 
!pip install matplotlib

### Load data and import libraries

If not already downloaded, the following cell will automatically download the processed data stored [here](https://osf.io/szmw6).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# importing rastermap
# (this will be slow the first time since it is compiling the numba functions)
from rastermap import Rastermap, utils
from scipy.stats import zscore

# download processed hippocampus recording from Grosmark & Buzsaki 2016
filename = utils.download_data(data_type="hippocampus")

dat = np.load(filename)

# spks is neurons by time
# (each timepoint is 200 ms)
spks = dat["spks"]
n_neurons, n_time = spks.shape
print(f"{n_neurons} neurons by {n_time} timepoints")
# zscore activity (each neuron activity trace is then mean 0 and standard-deviation 1)
spks = zscore(spks, axis=1)

# location of the rat and speed
loc2d = dat["loc2d"] # 2D location
loc_signed = dat["loc_signed"] # left runs are positive and right runs are negative
speed = (np.diff(loc2d, axis=0)**2).sum(axis=1)**0.5
speed = np.concatenate((np.zeros((1,)), speed), axis=0)

# which neurons in the recording are pyramidal cells
pyr_cells = dat["pyr_cells"].astype("int")

### Run Rastermap

Let's sort the single neurons with Rastermap, skipping clustering and upsampling:

In [ ]:
model = Rastermap(n_clusters=None, # None turns off clustering and sorts single neurons 
                  n_PCs=64, # use fewer PCs than neurons
                  locality=0.15, # some locality in sorting (this is a value from 0-1)
                  time_lag_window=15, # use future timepoints to compute correlation
                  grid_upsample=0, # 0 turns off upsampling since we're using single neurons
                ).fit(spks)
y = model.embedding # neurons x 1
isort = model.isort

compute tuning curves along linear corridor

In [ ]:
n_pos = 15
bins = np.arange(-1, 1+1./n_pos, 1./n_pos)
ibin = np.digitize(loc_signed, bins) - 1
n_bins = ibin.max()
inan = np.isnan(loc_signed)
ibin[inan] = -1
tcurves = np.zeros((spks.shape[0], n_bins))
for b in range(n_bins):
    tcurves[:, b] = spks[:, ibin==b].mean(axis=1)
tcurves -= tcurves.mean(axis=1, keepdims=True)

### Visualization

Use the Rastermap sorting to visualize the neural activity and tuning curves:

In [ ]:
# timepoints to visualize
xmin = 1000
xmax = xmin + 2000

# make figure with grid for easy plotting
fig = plt.figure(figsize=(12,8), dpi=200)
grid = plt.GridSpec(10, 24, figure=fig, wspace = 0.1, hspace = 0.4)

# plot location
ax = plt.subplot(grid[0, :-5])
ax.plot(loc2d[xmin:xmax])#, color=kp_colors[0])
ax.set_xlim([0, xmax-xmin])
ax.axis("off")
ax.set_title("2D location")

# plot running speed
ax = plt.subplot(grid[1, :-5])
ax.plot(speed[xmin:xmax], color=0.5*np.ones(3))
ax.set_xlim([0, xmax-xmin])
ax.axis("off")
ax.set_title("running speed")

# plot sorted neural activity
ax = plt.subplot(grid[2:, :-5])
ax.imshow(spks[isort, xmin:xmax], cmap="gray_r", vmin=0, vmax=1.2, aspect="auto")
ax.set_xlabel("time")
ax.set_ylabel("superneurons")

# excitatory cells in yellow, and inhibitory cells in dark blue
# (could replace this with a colorbar or other property)
ax = plt.subplot(grid[2:, -5])
ax.imshow(pyr_cells[isort, np.newaxis],
          cmap="viridis", aspect="auto")
ax.axis("off")

# plot single-neuron tuning curves
ax = plt.subplot(grid[2:, -4:])
x = np.arange(0, n_pos)
dy = 2
xpad = n_pos/10
nn = spks.shape[0]
for t in range(len(tcurves)):
    ax.plot(x, tcurves[isort[t], :n_pos]*dy - dy/2 + nn - t, 
            color="k", lw=0.5)
    ax.plot(x+n_pos+xpad, tcurves[isort[t], n_pos:]*dy - dy/2 + nn - t, 
            color="k", lw=0.5)
for j in range(2):
    xstr = "position\n(left run)" if j==0 else "position\n(right run)"
    ax.text(n_pos/2 + j*(n_pos+xpad), -14, xstr, ha="center")
    ax.text(j*(n_pos+xpad), -3, "0")
    ax.text(n_pos + j*(n_pos+xpad), -3, "1.6", ha="right")
ax.set_ylim([0, nn])
ax.axis("off")

### Settings

You can see all the rastermap settings with `Rastermap?`

In [ ]:
Rastermap?

### Outputs

All the attributes assigned to the Rastermap `model` are listed with `Rastermap.fit?`

In [ ]:
Rastermap.fit?